In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path

path = "/Users/Dipesh/OneDrive/Research_Engineer/Papers/M2EC/results/AWS_Google/"
#path = "/Users/Dipesh/OneDrive/Research_Engineer/Papers/M2EC/results/AWS_all_Google/"

bandwidth ={}
latency  = {}

bw = []
lt = []

listUse=['frankfurt_google', 'london_google', 'mumbai_google', 'northvirginia_google', 'oregon_google', 'saopaulo_google', 'singapore_google', 'sydney_google', 'tokyo_google']


In [154]:
dirList = os.listdir(path)
dirList.remove(".DS_Store")
#for i in listUse:
   # for j in listUse: 
for i in dirList:
    for j in dirList: 
        if i is j:
            bw.append(0.01)
            lt.append(0.01)
            continue
            

        # bandwidth
        if "_aws" in j:
            location = path+"%s/combined_bandwidth_aws_%s_%s.txt"%(i,j,i)
        else:
            location = path+"%s/combined_bandwidth_google_%s_%s.txt"%(i,j,i)
        
        my_file = Path(location)
        if my_file.exists()==False:
            bw.append(0)
            lt.append(0)
            continue

        df_bandwidth = pd.read_csv(location, sep='\s+', skiprows=3, nrows=100, header=None)
        headers = ["0", "1", "time","time_unit","transfer","transfer_unit", "bitrate", "bitrate_unit", "retr", "cwnd", "cwnd_unit"]
        df_bandwidth.columns = headers     
        df_bandwidth.drop(['0', '1', 'time', 'time_unit','transfer', 'transfer_unit','cwnd', 'cwnd_unit', 'retr'], axis=1, inplace= True)
        val = df_bandwidth["bitrate"].mean()
        
        bandwidth[j+"_"+i]=val
        bw.append(val)
  
        # latency
        if "_aws" in j:
            location = path+"%s/combined_latency_aws_%s_%s.txt"%(i,j,i)
        else:
            location = path+"%s/combined_latency_google_%s_%s.txt"%(i,j,i)
        
        df_lat = pd.read_csv(location, sep='\s+', skiprows=1, nrows=100, header=None)

        headers_lat = ["0", "1", "2", "3", "4", "5", "latency", "unit"]
        df_lat.columns = headers_lat
        df_lat.drop(['0', '1', '2', '3','4', '5'], axis=1, inplace= True)        
        df_lat["latency"] = df_lat["latency"].map(lambda x:x.replace("time=",""))
        df_lat["latency"] = df_lat["latency"].astype('float')
        #print(df_lat["latency"].max())
        val_lat = df_lat["latency"].mean()
        
        latency[j+"_"+i]=val_lat
        lt.append(val_lat)

In [155]:
# bandwidth
arrayBw = np.array(bw)
#arrayBw= arrayBw.reshape(-1,len(dirList))
arrayBw= arrayBw.reshape(-1,len(dirList))

max_bd = np.max(arrayBw)
min_bd = np.amin(arrayBw[arrayBw>0.01])

arrayBw = (arrayBw-min_bd)/(max_bd-min_bd)
arrayBw = arrayBw.round(4)
#print(latency)

In [156]:
# latency
arrayLt = np.array(lt)
arrayLt = arrayLt.reshape(-1,len(dirList))

max_lt = np.max(arrayLt)
min_lt = np.amin(arrayLt[arrayLt>0.01])

arrayLt = 1-(arrayLt-min_lt)/(max_lt-min_lt)

arrayLt = arrayLt.round(4)

In [157]:
arrayLt = np.where(arrayLt==1.0041, 0, arrayLt)

In [158]:
combined = (arrayBw+arrayLt)/2
print(combined.shape)

(18, 18)


In [159]:
df = pd.DataFrame(combined)
df

0        1        2        3        4        5        6        7   \
0   0.50015  0.90955  0.66940  0.60535  0.38910  0.07590  0.46055  0.43430   
1   0.81400  0.50015  0.59940  0.80980  0.20380  0.06760  0.41745  0.42460   
2   0.65505  0.62055  0.50015  0.99885  0.38360  0.08290  0.46745  0.45060   
3   0.51660  0.54260  0.94330  0.50015  0.20385  0.08920  0.44025  0.44595   
4   0.39510  0.21985  0.39105  0.22195  0.50015  0.95285  0.29540  0.22045   
5   0.05690  0.07720  0.07295  0.09120  0.91860  0.50015  0.15735  0.18680   
6   0.45085  0.43585  0.49395  0.46665  0.28735  0.17460  0.50015  0.95215   
7   0.39690  0.40630  0.44120  0.42825  0.20210  0.18635  0.88625  0.50015   
8   0.33580  0.35790  0.32925  0.36940  0.23995  0.24470  0.45870  0.49375   
9   0.32400  0.35240  0.34690  0.35450  0.20435  0.25785  0.46015  0.45615   
10  0.21990  0.24760  0.23200  0.25970  0.09080  0.02830  0.36585  0.36600   
11  0.23250  0.25355  0.24910  0.28225  0.04550  0.03280  0.35835  0.40635   
12  0.32045  0.14600  0.32585  0.15915  0.47170  0.47765  0.22800  0.25555   
13  0.14135  0.14300  0.13530  0.17240  0.48170  0.48075  0.24000  0.27445   
14  0.15995  0.17245  0.16605  0.18545  0.19385  0.34940  0.27335  0.28285   
15  0.16335  0.15060  0.16725  0.17035  0.31155  0.33065  0.25795  0.27245   
16  0.20405  0.23495  0.23260  0.24995  0.36225  0.36535  0.32960  0.33940   
17  0.21060  0.23740  0.23595  0.26550  0.37325  0.37195  0.33060  0.37105   

         8        9        10       11       12       13       14       15  \
0   0.33030  0.34755  0.24240  0.24410  0.32995  0.13885  0.15975  0.16460   
1   0.33265  0.35860  0.22675  0.22960  0.14160  0.15725  0.14660  0.17330   
2   0.36670  0.36835  0.25680  0.26700  0.33160  0.16895  0.16600  0.18625   
3   0.34795  0.36885  0.24285  0.24585  0.15870  0.17295  0.16100  0.19265   
4   0.24120  0.22210  0.11055  0.05885  0.48300  0.46735  0.20080  0.32300   
5   0.22565  0.27625  0.01535  0.01425  0.48430  0.48520  0.31140  0.36880   
6   0.45800  0.48780  0.36580  0.37140  0.22365  0.26475  0.26905  0.28470   
7   0.46555  0.51415  0.34005  0.34165  0.24805  0.27380  0.25300  0.29635   
8   0.50015  0.65975  0.30225  0.29440  0.32535  0.33315  0.36560  0.32130   
9   0.59505  0.50015  0.28580  0.28570  0.33590  0.35265  0.30005  0.34620   
10  0.29010  0.30630  0.50015  0.90165  0.10295  0.10680  0.07505  0.13060   
11  0.27270  0.33425  0.68440  0.50015  0.09845  0.10980  0.11180  0.13770   
12  0.32550  0.33515  0.10340  0.10310  0.50015  0.90435  0.29875  0.41255   
13  0.31145  0.36880  0.09120  0.09005  0.75220  0.50015  0.39170  0.48740   
14  0.36835  0.32590  0.11325  0.13105  0.28960  0.42700  0.50015  0.96740   
15  0.30005  0.33740  0.11190  0.11080  0.43455  0.44545  0.66635  0.50015   
16  0.39040  0.42660  0.15985  0.19675  0.47165  0.46415  0.38500  0.40270   
17  0.39925  0.46695  0.17120  0.17250  0.47845  0.51420  0.36145  0.43825   

         16       17  
0   0.21870  0.21270  
1   0.20675  0.24245  
2   0.23280  0.25295  
3   0.22045  0.24175  
4   0.35845  0.36975  
5   0.34065  0.36500  
6   0.32825  0.34290  
7   0.31170  0.37260  
8   0.42925  0.41850  
9   0.39470  0.40465  
10  0.16940  0.18630  
11  0.16615  0.19180  
12  0.46175  0.45790  
13  0.42115  0.44755  
14  0.37415  0.39720  
15  0.36795  0.36875  
16  0.50015  0.79925  
17  0.63460  0.50015

In [160]:
#df = pd.DataFrame(combined, index=dirList, columns=dirList)
df = pd.DataFrame(combined)
df.columns = dirList
new_column = dirList
df.insert(0, column="From", value=new_column)

df = df.round(4)
df = df.replace(-0.0018,0)
df = df.replace(0.5002,1)
#df.replace(-0.0160,0)
df


From  frankfurt_aws  frankfurt_google  london_aws  \
0          frankfurt_aws         1.0000            0.9096      0.6694   
1       frankfurt_google         0.8140            1.0000      0.5994   
2             london_aws         0.6550            0.6205      1.0000   
3          london_google         0.5166            0.5426      0.9433   
4             mumbai_aws         0.3951            0.2198      0.3910   
5          mumbai_google         0.0569            0.0772      0.0730   
6      northvirginia_aws         0.4508            0.4358      0.4940   
7   northvirginia_google         0.3969            0.4063      0.4412   
8             oregon_aws         0.3358            0.3579      0.3292   
9          oregon_google         0.3240            0.3524      0.3469   
10          saopaulo_aws         0.2199            0.2476      0.2320   
11       saopaulo_google         0.2325            0.2536      0.2491   
12         singapore_aws         0.3204            0.1460      0.3259   
13      singapore_google         0.1414            0.1430      0.1353   
14            sydney_aws         0.1599            0.1724      0.1660   
15         sydney_google         0.1634            0.1506      0.1672   
16             tokyo_aws         0.2040            0.2350      0.2326   
17          tokyo_google         0.2106            0.2374      0.2360   

    london_google  mumbai_aws  mumbai_google  northvirginia_aws  \
0          0.6053      0.3891         0.0759             0.4606   
1          0.8098      0.2038         0.0676             0.4174   
2          0.9988      0.3836         0.0829             0.4674   
3          1.0000      0.2038         0.0892             0.4402   
4          0.2220      1.0000         0.9528             0.2954   
5          0.0912      0.9186         1.0000             0.1574   
6          0.4666      0.2874         0.1746             1.0000   
7          0.4282      0.2021         0.1863             0.8862   
8          0.3694      0.2400         0.2447             0.4587   
9          0.3545      0.2044         0.2579             0.4602   
10         0.2597      0.0908         0.0283             0.3658   
11         0.2822      0.0455         0.0328             0.3584   
12         0.1592      0.4717         0.4776             0.2280   
13         0.1724      0.4817         0.4808             0.2400   
14         0.1854      0.1938         0.3494             0.2734   
15         0.1704      0.3116         0.3306             0.2580   
16         0.2500      0.3622         0.3653             0.3296   
17         0.2655      0.3732         0.3720             0.3306   

    northvirginia_google  oregon_aws  oregon_google  saopaulo_aws  \
0                 0.4343      0.3303         0.3475        0.2424   
1                 0.4246      0.3326         0.3586        0.2268   
2                 0.4506      0.3667         0.3683        0.2568   
3                 0.4460      0.3480         0.3688        0.2428   
4                 0.2204      0.2412         0.2221        0.1106   
5                 0.1868      0.2256         0.2762        0.0154   
6                 0.9522      0.4580         0.4878        0.3658   
7                 1.0000      0.4656         0.5142        0.3400   
8                 0.4938      1.0000         0.6598        0.3022   
9                 0.4562      0.5950         1.0000        0.2858   
10                0.3660      0.2901         0.3063        1.0000   
11                0.4064      0.2727         0.3342        0.6844   
12                0.2556      0.3255         0.3352        0.1034   
13                0.2745      0.3114         0.3688        0.0912   
14                0.2828      0.3684         0.3259        0.1132   
15                0.2725      0.3000         0.3374        0.1119   
16                0.3394      0.3904         0.4266        0.1599   
17                0.3710      0.3992         0.4670        0.1712   

    saopaulo_google  singapore_aws  singapore_

In [161]:
#df.max(axis=1)

In [162]:
df.to_csv("combined.csv", sep=",")

In [163]:
new_df = pd.melt(df, id_vars=['From'], value_vars=df.columns.values.tolist()[1:],
    var_name='To', value_name='Value')

In [164]:
new_df.head()

From             To   Value
0     frankfurt_aws  frankfurt_aws  1.0000
1  frankfurt_google  frankfurt_aws  0.8140
2        london_aws  frankfurt_aws  0.6550
3     london_google  frankfurt_aws  0.5166
4        mumbai_aws  frankfurt_aws  0.3951

In [165]:
new_df.to_csv("values.csv", sep=",")

In [166]:
dirList

['frankfurt_aws',
 'frankfurt_google',
 'london_aws',
 'london_google',
 'mumbai_aws',
 'mumbai_google',
 'northvirginia_aws',
 'northvirginia_google',
 'oregon_aws',
 'oregon_google',
 'saopaulo_aws',
 'saopaulo_google',
 'singapore_aws',
 'singapore_google',
 'sydney_aws',
 'sydney_google',
 'tokyo_aws',
 'tokyo_google']